In [1]:
import os

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '../'))
os.chdir(project_root)

import sys
sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset

from src.config import CCY_PAIRS, START_DATE, END_DATE
from src.data_utils import get_fx_data, compute_log_ret, to_binary, to_float, get_min_max_info
from src.models.gan import GAN, preprocess_data

In [19]:
df = get_fx_data(CCY_PAIRS, START_DATE, END_DATE, df=True)
log_ret_df = compute_log_ret(df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [20]:
log_ret_tensor = preprocess_data(log_ret_df)

In [37]:
# Define parameters
latent_dim = 100
output_dim = log_ret_tensor.shape[1]
epochs = 200
batch_size = 128

In [38]:
data_loader = DataLoader(TensorDataset(log_ret_tensor), batch_size=batch_size, shuffle=True)

In [41]:
gan = GAN(latent_dim, output_dim)

In [42]:
gan.fit(data_loader, epochs, batch_size)

Epoch [0/200], Batch [28/29], D Loss: 0.8731, G Loss: 0.7128
Epoch [1/200], Batch [28/29], D Loss: 0.8111, G Loss: 0.6970
Epoch [2/200], Batch [28/29], D Loss: 0.7841, G Loss: 0.6963
Epoch [10/200], Batch [28/29], D Loss: 0.5525, G Loss: 0.9354
Epoch [20/200], Batch [28/29], D Loss: 0.2629, G Loss: 1.5613
Epoch [30/200], Batch [28/29], D Loss: 0.1316, G Loss: 2.2224
Epoch [40/200], Batch [28/29], D Loss: 0.0661, G Loss: 2.8889
Epoch [50/200], Batch [28/29], D Loss: 0.0398, G Loss: 3.3728
Epoch [60/200], Batch [28/29], D Loss: 0.0236, G Loss: 3.9049
Epoch [70/200], Batch [28/29], D Loss: 0.0158, G Loss: 4.2676
Epoch [80/200], Batch [28/29], D Loss: 0.0113, G Loss: 4.5314
Epoch [90/200], Batch [28/29], D Loss: 0.0085, G Loss: 4.9031
Epoch [100/200], Batch [28/29], D Loss: 0.0061, G Loss: 5.2548
Epoch [110/200], Batch [28/29], D Loss: 0.0044, G Loss: 5.5828
Epoch [120/200], Batch [28/29], D Loss: 0.0037, G Loss: 5.7117
Epoch [130/200], Batch [28/29], D Loss: 0.0028, G Loss: 5.9948
Epoch [

In [43]:
generated_samples = gan.generate_samples(1000)

In [44]:
import pangenerated_samples

array([[ 0.54090154,  0.46606746, -0.13051216, ...,  0.26923588,
        -0.09128861,  0.43420127],
       [ 0.96547794,  0.99863696,  0.9865766 , ...,  0.55221313,
        -0.57674545,  0.78698367],
       [ 0.82606834,  0.8267312 ,  0.16979527, ...,  0.42192906,
        -0.4525606 ,  0.6440134 ],
       ...,
       [ 0.67772   ,  0.8078917 ,  0.61306846, ...,  0.24821432,
        -0.23107502,  0.29164436],
       [ 0.0815408 ,  0.07602684, -0.41747794, ...,  0.49129   ,
         0.30396533,  0.5869345 ],
       [ 0.8156811 ,  0.9638678 ,  0.8339657 , ...,  0.41049105,
        -0.2883414 ,  0.5227554 ]], dtype=float32)